In [1]:
from datetime import datetime

import ee
import geopandas as gpd
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
from descarteslabs.geo import DLTile
from tqdm import tqdm

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [14]:
def create_tiles(region, tilesize, padding):
    tiles = DLTile.iter_from_shape(region, tilesize=tilesize, resolution=10, pad=padding)
    tiles = [tile for tile in tiles]
    return tiles

import concurrent.futures

def get_image_data(tiles, start_date, end_date, clear_threshold=0.6):
    # Harmonized Sentinel-2 Level 2A collection.
    s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

    # Cloud Score+ image collection. Note Cloud Score+ is produced from Sentinel-2
    # Level 1C data and can be applied to either L1C or L2A collections.
    csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
    QA_BAND = 'cs_cdf'

    # Make a clear median composite.
    composite = (s2
        .filterDate(start_date, end_date)
        .linkCollection(csPlus, [QA_BAND])
        .map(lambda img: img.updateMask(img.select(QA_BAND).gte(clear_threshold)))
        .median())
    
    data = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Process each tile in parallel
        futures = [executor.submit(process_tile, tile, composite) for tile in tiles]
        
        # Collect the results as they become available
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            data.append(result)
    
    return data

def process_tile(tile, composite):
    tile_geom = ee.Geometry.Rectangle(tile.geometry.bounds)
    composite_tile = composite.clipToBoundsAndScale(
        geometry=tile_geom,
        width=tile.tilesize + 2,
        height=tile.tilesize + 2)
    npy = ee.data.computePixels({
        'bandIds': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B8', 'B9', 'B11', 'B12'],
        'expression': composite_tile,
        'fileFormat': 'NUMPY_NDARRAY',
        'grid': {'crsCode': tile.crs},
    })
    
    return npy

def pad_patch(patch, height, width=None):
    """
    Depending on how a polygon falls across pixel boundaries, the resulting patch can be slightly
    bigger or smaller than intended.
    pad_patch trims pixels extending beyond the desired number of pixels if the
    patch is larger than desired. If the patch is smaller, it will fill the
    edge by reflecting the values.
    If trimmed, the patch should be trimmed from the center of the patch.
    """
    if width is None:
        width = height
    
    # convert from a structured array to a numpy array
    #patch_dtypes = patch.dtype
    patch = np.array(patch.tolist())
    patch_height, patch_width,_ = patch.shape
    
    if patch_height > height:
        trim_top = (patch_height - height) // 2
        trim_bottom = trim_top + height
    else:
        trim_top = 0
        trim_bottom = patch_height
    
    if patch_width > width:
        trim_left = (patch_width - width) // 2
        trim_right = trim_left + width
    else:
        trim_left = 0
        trim_right = patch_width
    
    trimmed_patch = patch[trim_top:trim_bottom, trim_left:trim_right, :]
    
    if patch_height < height:
        pad_top = (height - patch_height) // 2
        pad_bottom = pad_top + patch_height
        padded_patch = np.pad(trimmed_patch, ((pad_top, height - pad_bottom), (0, 0), (0, 0)), mode='reflect')
    else:
        padded_patch = trimmed_patch
    
    if patch_width < width:
        pad_left = (width - patch_width) // 2
        pad_right = pad_left + patch_width
        padded_patch = np.pad(padded_patch, ((0, 0), (pad_left, width - pad_right), (0, 0)), mode='reflect')
    # convert back to a structured array where each band is named
    #padded_patch = np.array(padded_patch, dtype=patch_dtypes)
    return padded_patch

def chips_from_tile(data, tile, width, stride):
    """
    Break a larger tile of Sentinel data into a set of patches that
    a model can process.
    Inputs:
        - data: Sentinel data. Typically a numpy masked array
        - tile_coords: bounds of the tile in the format (west, south, east, north)
        - stride: number of pixels between each patch
    Outputs:
        - chips: A list of numpy arrays of the shape the model requires
        - chip_coords: A list of shapely polygon features describing the patch bounds
    """
    (west, south, east, north) = tile.bounds
    delta_x = east - west
    delta_y = south - north
    x_per_pixel = delta_x / np.shape(data)[0]
    y_per_pixel = delta_y / np.shape(data)[1]

    # The tile is broken into the number of whole patches
    # Regions extending beyond will not be padded and processed
    chip_coords = []
    chips = []

    # Extract patches and create a shapely polygon for each patch
    for i in range(0, np.shape(data)[0] - width + stride, stride):
        for j in range(0, np.shape(data)[1] - width + stride, stride):
            patch = data[j : j + width, i : i + width]
            chips.append(patch)

            nw_coord = [west + i * x_per_pixel, north + j * y_per_pixel]
            ne_coord = [west + (i + width) * x_per_pixel, north + j * y_per_pixel]
            sw_coord = [west + i * x_per_pixel, north + (j + width) * y_per_pixel]
            se_coord = [west + (i + width) * x_per_pixel, north + (j + width) * y_per_pixel]
            tile_geometry = [nw_coord, sw_coord, se_coord, ne_coord, nw_coord]
            chip_coords.append(shapely.geometry.Polygon(tile_geometry))
    chip_coords = gpd.GeoDataFrame(geometry=chip_coords, crs=tile.crs)
    return chips, chip_coords

def unit_norm(samples):
    """
    Channel-wise normalization of pixels in a patch.
    Means and deviations are constants generated from an earlier dataset.
    If changed, models will need to be retrained
    Input: (n,n,12) numpy array or list.
    Returns: normalized numpy array
    """
    means = [1405.8951, 1175.9235, 1172.4902, 1091.9574, 1321.1304, 2181.5363, 2670.2361, 2491.2354, 2948.3846, 420.1552, 2028.0025, 1076.2417]
    deviations = [291.9438, 398.5558, 504.557, 748.6153, 651.8549, 730.9811, 913.6062, 893.9428, 1055.297, 225.2153, 970.1915, 752.8637]
    normalized_samples = np.zeros_like(samples).astype('float32')
    for i in range(0, 12):
        #normalize each channel to global unit norm
        normalized_samples[:,:,i] = (np.array(samples.astype(float))[:,:,i] - means[i]) / deviations[i]
    return normalized_samples

In [16]:
model = keras.models.load_model('../models/44px_v2.9_2022-02-28.h5')

region = gpd.read_file('../data/boundaries/test_region.geojson').geometry[0].__geo_interface__

tile_size = 528
tile_padding = 16

chip_width = 44
chip_stride = 11

start_date = datetime(2023, 5, 1)
end_date = datetime(2023, 11, 1)
clear_threshold = 0.6

In [49]:
# start a timer
start = datetime.now()
tiles = create_tiles(region, tile_size, tile_padding)
print(f"{len(tiles)} tiles have been created")

data = get_image_data(tiles, start_date, end_date, clear_threshold)
data = np.array([pad_patch(patch, tile_size) for patch in data])
data_norm = np.array([unit_norm(d) for d in data])

# convert the tiles to chips
chip_stack = []
chip_geom_stack = []
for tile_data, tile_geom in zip(data_norm, tiles):
    chips, chip_geoms = chips_from_tile(tile_data, tile_geom, chip_width, chip_stride)
    chip_stack.extend(chips)
    chip_geom_stack.append(chip_geoms)

chip_stack = np.array(chip_stack)
chip_geom_stack = gpd.GeoDataFrame(pd.concat(chip_geom_stack, ignore_index=True), crs=tiles[0].crs)
print(f"Tiles have been broken into {len(chip_stack):,.0f} chips")

# make predictions and create a gdf
preds = model.predict(chip_stack)[:,1]
pred_df = gpd.GeoDataFrame(chip_geom_stack, crs=tiles[0].crs)
pred_df['preds'] = preds

# end the timer
end = datetime.now()

# print the time it took to run the pipeline
area_m2 = len(tiles) * (tile_size * 10) ** 2
# convert the meters squared to hectares
area_ha = area_m2 / 10000
duration = end - start
minutes, seconds = divmod(duration.total_seconds(), 60)
print(f"{area_ha:,.0f} hectares were analyzed in {minutes:.0f} minutes and {seconds:.0f} seconds")
print(f"At this speed, you could process an area the size of Rhode Island in {313900 * duration.total_seconds() / area_ha:.0f} seconds")
minutes, seconds = divmod(2203 * 313900 * duration.total_seconds() / area_ha, 60)
print(f"and the Amazon basin in {minutes / 60:,.1f} hours ({minutes / 60 / 24:,.1f} days)")

49 tiles have been created
Tiles have been broken into 99,225 chips
3101/3101 [==============================] - 7s 2ms/step
136,604 hectares were analyzed in 1 minutes and 18 seconds
At this speed, you could process an area the size of Rhode Island in 180 seconds
and the Amazon basin in 110.3 hours (4.6 days)


In [ ]:
# plot the chips in chip_stack where the preds are above a threshold
threshold = 0.9
positive_indices = np.where(preds > threshold)[0]
positive_chips = chip_stack[positive_indices]
print(len(positive_chips), 'chips with predictions above', threshold)
num_plots = int(np.ceil(np.sqrt(len(positive_chips))))
fig, axs = plt.subplots(num_plots, num_plots, figsize=(10,10))
for i, ax in enumerate(axs.flatten()):
    if i < len(positive_chips):
        ax.imshow(np.clip((positive_chips[i][:,:,(3,2,1)] + 2) / 5, 0, 1))
        ax.set_axis_off()
    else:
        ax.set_visible(False)
plt.show()